In [1]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 5.5MB/s 


In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import datetime
import os
import numpy as np

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
## ADDITIONAL FUNCTIONS
if not os.path.exists('models'):
    os.makedirs('models')
    
pStopWordsList = stopwords.words('english') + stopwords.words('russian')

def convert_amount(pAmount, rubles = False):
    try:
        if rubles:
            cAmount = pytils.numeral.rubles(abs(float(pAmount)), zero_for_kopeck=True)    
        else:
            cAmount = pytils.numeral.in_words(abs(int(pAmount)))    
    except Exception as e:
        cAmount = pAmount  
    return cAmount

def convert_date(pDate, format_mode, formatDate = None):
    formatDates = ['%Y-%m-%d',
                   '%d-%m-%Y',
                   '%d/%m/%Y',
                   '%d.%m.%Y',
                   '%y-%m-%d',
                   '%d-%m-%y',
                   '%d/%m/%y',
                   '%d.%m.%y',
                   '%Y-%m-%d %H:%M:%S',
                   '%d-%m-%Y %H:%M:%S',
                   '%d/%m/%Y %H:%M:%S',
                   '%d.%m.%Y %H:%M:%S',
                   '%y-%m-%d %H:%M:%S',
                   '%d-%m-%y %H:%M:%S',
                   '%d/%m/%y %H:%M:%S',
                   '%d.%m.%y %H:%M:%S']
    if formatDate!=None:
        cDate = datetime.datetime.strptime(pDate, formatDate).date()
    else:
        for fd in formatDates:
            try:
                cDate = datetime.datetime.strptime(pDate, fd).date()
                break
            except Exception as e:
                cDate=None
    if cDate == None:
        print('convert_date: Can not define date format for "%s"' % pDate)
        return ''
    if format_mode == 1:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year - 2000)
    if format_mode == 2:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '{} {} {}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)
    if format_mode == 3:
        dmonth = {1: 'января', 2: 'февраля', 3: 'марта', 4: 'апреля', 5: 'мая', 6: 'июня', 7: 'июля', 8: 'августа', 9: 'сентября', 10: 'октября', 11: 'ноября', 12: 'декабря'}   
        dformat = '"{}" {} {}'
        res = dformat.format(str(cDate.day).zfill(2), dmonth[cDate.month], cDate.year)            
    if format_mode == 4:
        dmonth = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7: '07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
        dformat = '{}.{}.{}'
        res = dformat.format(cDate.day, dmonth[cDate.month], cDate.year)            
    return res

def checkdate(text, pattern):
    res = ''
    if pattern==None: return ''
    text = text.lower()
    pattern = pattern.lower()
    res = pattern
    if res in text: return res
    res = convert_date(pattern, format_mode=1).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=4).lower()
    if res in text: return res   
    res = res.replace('.','/')
    if res in text: return res   
    res = res.replace('/','-')
    if res in text: return res       
    res = convert_date(pattern, format_mode=2).lower()
    if res in text: return res
    res = convert_date(pattern, format_mode=3).lower()
    if res in text: return res
    res = res[1:]
    if res in text: return res
    res = res.replace('""', '')
    if res in text: return res
    
    if " ".join([w for w in convert_date(pattern, format_mode=1).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=1)])
    if " ".join([w for w in convert_date(pattern, format_mode=2).lower()]) in text:
        res = " ".join([w for w in convert_date(pattern, format_mode=2)])
    
    return ''
    
def checkamount(text, pattern, space =3):
    digits = ["0","1","2","3","4","5","6","7","8","9"]
    if pattern in (None,''): return ''
    res = ''
    
    if pattern in (None,'','nan'): return ''
    
    if pattern == 'nan':
        return res
    if pattern in text:
        res = pattern
    res=pattern
    if res in text: return res
    
    pattern = pattern.lower().replace(' ','').replace(',', '.')#.replace('-', '')
    res = pattern
    if res in text: return res
    
    res = pattern.replace('.', ',')
    if res in text: return res
    
    res = pattern.replace(',', '.')
    if res in text: return res

    res = pattern.replace(',', '-')
    if res in text: return res

    res = pattern.replace('.', '-')
    if res in text: return res
    
    try:
        res = '{:_}'.format(float(pattern)).replace('_', ' ')
        if res in text: return res
        res = res.replace('.',',')
        if res in text: return res
        
        res = res.replace('.','-')
        if res in text: return res
    except Exception as e:
        tmp = None
    
    res = convert_amount(pattern).lower()
    if convert_amount(pattern, rubles=True).lower() in text:
    
        res = convert_amount(pattern, rubles=True).lower()        
    if res in text: return res
        
    tmp = pattern.split('.')
    res = convert_amount(tmp[0]).lower()
    tmp[0] = '{:,}'.format(int(tmp[0])).replace(',', ' ')
    
    res = ','.join(tmp)
    if res in text: return res
    
    res = '.'.join(tmp)
    if res in text: return res
    
    reg = ''
    for l in pattern:
        if l in digits:
            reg = '%s%s.{0,%s}' % (reg, l,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]

    for w in pattern.split():
        reg = '%s%s.{0,%s}' % (reg, w,space)
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-space]
    
    return ''

def checkstring(text, target, spaces = 2):
    if target in (None,'','nan'): return ''
    text = text.lower()
    target = target.lower()
    '''
    reg = ''
    for l in target:
        reg = '%s%s.{0,%s}' % (re.escape(reg), l,spaces)
    
    m = re.search(reg, text)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    '''
    target = target.replace('ооо','').replace('ао бывш.зао', '').\
             replace('зао','').replace('оао','').replace('ао','').\
             replace('ип','').replace(',','').strip()
    if target in text:
        return target
    if target in ['бн', 'б/н']:
        return ''
    if target.replace('0','o') in text.replace('0','o'):
        return target.replace('0','o')
    if target.replace('з','3') in text.replace('з','3'):
        return target.replace('з','3')    
    if target.replace('э','3') in text.replace('э','3'):
        return target.replace('э','3')    
    if target.replace('э','9') in text.replace('э','9'):
        return target.replace('э','9')    
    if target.replace('б','6') in text.replace('б','6'):
        return target.replace('б','6')    
    for ch in [' ','_', '\\','`','*','{','}','[',']','(',')','>','#','+','-','.','!','$','\'', ' ']:
        if ch in target:
            target = target.replace(ch,"")
            if target in text:
                return target
    return ''

def process_text_layer(TextLayer, pStopWordsList = []):
    txt =  " ".join([w for w in TextLayer.lower().split() \
                     if (not w in pStopWordsList)])

    txt = txt. \
        replace('«', ''). \
        replace('»', ''). \
        replace('(', ''). \
        replace(')', ''). \
        replace('\[)', ''). \
        replace('\]', ''). \
        replace('^', ''). \
        replace('\\', '')

    return txt

def tagging_sentence_by_targets(ptag, sentence_tagged, df_targets, pmodel_name):
    ret = sentence_tagged
    for r in df_targets[df_targets['tag']==ptag]['string_value']:
        tagging_sentence(ret, r, pmodel_name)
    
    return ret

def word2features2(sent, i, exclude = 'O', bias=3):
    word = sent[i][0]
    if sent[i][2] == exclude:
        return None

    features = {
        'bias': bias,
        'word.lower()': word.lower(),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'BOS': False,
        'EOS': False
    }
    for w in range(1, bias+1):
        if i > w:
            word1 = sent[i-w][0]
            features.update({
                '-%s:word.lower()' % (w): word1.lower()
            })
            
    if i < 1:
        features['BOS'] = True

    for w in range(1, bias+1):
        if i < len(sent)-w:
            word1 = sent[i+w][0]
            features.update({
                '+%s:word.lower()' % (w): word1.lower()
            })
            
    if i >= len(sent)-1:
        features['EOS'] = True

    return features

def sent2features(sent, exclude = 'O'):
    result = list()
    for i in range(len(sent)):
        ret = word2features2(sent, i, exclude = exclude, bias=10)
        if ret !=None:
            result.append(ret)
    return result

def sent2labels(sent, exclude = 'O'):
    result = list()
    for token, postag, label in sent:
        if label !=exclude:
            result.append(label)        
    return result

def tagging_sentence(sentence, target, tag):
    sentence_text=  ' '.join([w['Word'] for w in sentence])
    r_tag = ''

    try:
        res_find  = re.finditer(target.lower(), sentence_text.lower())
    except:
        return sentence

    for m in res_find:
        r_tag = 'B-%s' % tag
        for i in range(len(sentence_text[:m.start()].split()), len(sentence_text[:m.end()].split())):
            """
            if r_tag==('B-%s' % tag):
                if i>0:
                    sentence[i-1]['Tag']= 'B1-%s' % tag
                if i>1:
                    sentence[i-2]['Tag']= 'B2-%s' % tag                      
                if i>2:
                    sentence[i-3]['Tag']= 'B3-%s' % tag
                if i>3:
                    sentence[i-4]['Tag']= 'B4-%s' % tag  
            """
            sentence[i]['Tag']= r_tag
            r_tag = 'I-%s' % tag   
            """
            for j in range(1,5):
                if i+j < len(sentence):
                    sentence[i+j]['Tag']='E%s-%s' % (str(j), tag)
            """
    return sentence

def prepare_sentence(num, sentence):
    words = sentence.replace('"', '').split()
    tmp = []
    for w in words:
        tmp.append({'Sentence #': 'Sentence: %s' % num, 'Word' : w, 'POS' : 'NN', 'Tag': 'O'})
    return tmp

In [5]:
!gdown --id 1hYI4ovDNu2jUUBOMAEKya6BPbYEBd5An

Downloading...
From: https://drive.google.com/uc?id=1hYI4ovDNu2jUUBOMAEKya6BPbYEBd5An
To: /content/NER PAD.csv
41.8MB [00:00, 81.1MB/s]


In [6]:
data = pd.read_csv('NER PAD.csv')
for column in data.columns:
    data[column] = data[column].astype(str)
data['DOCCPTYINN'] = data['DOCCPTYINN'].apply(lambda r: r[:-2] if type(r) == str else r)

In [7]:
PADData = data.copy()
PADData['string_value'] = PADData.apply(lambda r: process_text_layer(r['string_value'], pStopWordsList), axis = 1)

In [8]:
PADData.head()

,tag,DOCTYPE,DOCNUM,DOCDATE,DOCAMOUNT,DOCCPTY,DOCAGRNUM,DOCAGRDATE,DOCCUSTOMER,DOCCPTYINN,string_value
0,SV990125172_ОРГ__INM_18'09'58_1.pdf,УПД2,(нацио,2018-12-31,1200,"Телеком ЛТД, ООО",П-1856у,2014-12-01,ТВ ДАРЬЯЛ,21190038,l * универсальный приложение № 1 постановлению...
1,SS528581_ОРГ__INM_17'05'39_A.pdf,АКТ,0000-000346,2019-04-10,10000,"РИА КАЛМЫКИЯ, АУ РК",0000-000290,2019-04-10,МИЦ ИЗВЕСТИЯ ООО,08160051,l лист согласования стр. 1 1 2 9 * 0 4 сохрани...
2,SF435712_ОРГ__INM_15'25'41_A.pdf,Счет-фактура,0000-004358,2019-03-31,"5273,76",ГОРМОСТ ГБУ,291-2013/ИК/А-158/13.,2013-04-01,АКЦЕПТ (ТЕЛЕВИЗИОННЫЙ КАНАЛ РЕН ТВ) ООО,77227654,l приложение № постановлению правительства рф ...
3,SV990124943_ОРГ__INM_19'00'43_A.pdf,АКТ,0000-012544,2019-06-30,1000,СТАТУС АО,Основной договор (Счет №А7407),2016-04-20,СПОРТ-ЭКСПРЕСС АО,77071792,"l акционерное общество ""регистраторское общест..."
4,S29013419_ОРИГИНАЛ__IND_1_22.05.12'21'36.pdf,АКТ,00000000014,2019-04-30,11800,СТАНДАРТ БЕЗОПАСНОСТИ ООО,04-14/ТРК-863/13СЖ,2013-12-13,Телерадиокомпания Петербург АО,78133587,"l * 000""стандарт безопасности"" адрес: 197376, ..."


In [9]:
models = ['DOCNUM',
            'DOCDATE',
            'DOCAMOUNT',
            'DOCCPTY',
            'DOCAGRNUM',
            'DOCAGRDATE',
            'DOCCUSTOMER',
            'DOCCPTYINN']

datasets = {}
for model_name in models:
    ds = PADData[['tag','string_value', model_name]].rename(columns={'string_value': 'x', model_name: 'target'})
    if 'DATE' in model_name:
        ds['target'] = ds.apply(lambda r: checkdate(r['x'], r['target']),axis=1).copy()
    if model_name in ['DOCNUM', 'DOCAGRNUM', 'DOCCUSTOMER', 'DOCCPTY']:
        ds['target'] = ds.apply(lambda r: checkstring(r["x"], r['target']),axis=1).copy()      
    if 'AMOUNT' in model_name:
        ds['target'] = ds.apply(lambda r: checkamount(r['x'], r['target']),axis=1).copy()
        
    ds = ds[ds['target']!=''].drop_duplicates().copy()
    ds["x_tagged"] = ds.apply(lambda r: prepare_sentence(r.name, r["x"]), axis=1)
    ds["x_tagged"] = ds.apply(lambda r: tagging_sentence(r['x_tagged'], r['target'], model_name), axis=1)
    
    datasets[model_name] = ds.copy()

In [10]:
def choose(word, x, y):
    x = x.strip()
    y = y.strip()
    word = word.strip()
    nx = len(x)
    ny = len(y)
    n = len(word)

    find_x = -1
    find_y = -1
    if x.find(word) != -1:
        find_x = max(len(word), find_x)
    if word.find(x) != -1:
        find_x = max(len(x), find_x)
    if y.find(word) != -1:
        find_y = max(len(word), find_y)
    if word.find(y) != -1:
        find_y = max(len(y), find_y)

    if find_x != -1 or find_y != -1:
        if find_x >= find_y:
            return 0
        else:
            return 1
    
    index = 0
    while index < nx and index < ny and index < n:
        if x[index] != word[index] and y[index] == word[index]:
            return 1
        elif x[index] == word[index] and y[index] != word[index]:
             return 0
        index += 1

    if index < ny - 1 and index < n - 1 and y[index + 1] == word[index + 1]:
        return 1
    elif index < nx - 1 and index < n - 1 and x[index + 1] == word[index + 1]:
        return 0
    
    return 0 #default

def merge_tags(row):
    x = row.tags_x
    y = row.tags_y
    if type(y) != list:
        return
    if type(x) != list:
        x = y
        return
    assert len(x) == len(y)
    for i in range(len(x)):
        if x[i] == 'O' and y[i] != 'O':
            x[i] = y[i]
        if x[i] != 'O' and y[i] != 'O' and x[i] != y[i]: #labels conflict
            print(f'Labels conflict with word {row.words_x[i]} --- {x[i]} : {row.target_x}; {y[i]} : {row.target_y}')
            if pd.isnull(row.target_x):
                x[i] = y[i]
                continue
            if pd.isnull(row.target_y):
                continue
            choosed = choose(row.words_x[i], row.target_x, row.target_y)
            if choosed == 0:
                print(f'Selected : {row.target_x}')
            else:
                print(f'Selected : {row.target_y}')
                x[i] = y[i]

In [11]:
d_merged = datasets["DOCNUM"].copy()
d_merged["tags"] = d_merged.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
d_merged["words"] = d_merged.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
d_merged.head(1)

,tag,x,target,x_tagged,tags,words
2,SF435712_ОРГ__INM_15'25'41_A.pdf,l приложение № постановлению правительства рф ...,0000-004358,"[{'Sentence #': 'Sentence: 2', 'Word': 'l', 'P...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, O, O, O,...","[l, приложение, №, постановлению, правительств..."


In [12]:
for model in models[1:]:
    d_new = datasets[model].copy()
    d_new["tags"] = d_new.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
    d_new["words"] = d_new.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])
    d_new.head(1)
    d_merged = pd.merge(d_merged, d_new.loc[:, ["tags", "target", "words", "tag"]], left_index=True, right_index=True, how="outer")
    d_merged.loc[pd.isna(d_merged.tags_x), 'tags_x'] = d_merged.loc[pd.isna(d_merged.tags_x), 'tags_y']
    d_merged.loc[pd.isna(d_merged.words_x), 'words_x'] = d_merged.loc[pd.isna(d_merged.words_x), 'words_y']
    d_merged.loc[pd.isna(d_merged.tag_x), 'tag_x'] = d_merged.loc[pd.isna(d_merged.tag_x), 'tag_y']
    d_merged.apply(merge_tags, axis=1)
    d_merged.drop(columns=['target_y', 'tags_y', 'words_y', 'tag_y'], inplace=True)
    d_merged.rename(columns={'target_x' : 'target', 'tags_x' : 'tags', 'words_x' : 'words', 'tag_x' : 'tag'}, inplace=True)

Labels conflict with word 16.04.2019 --- B-DOCNUM : 1; B-DOCDATE : 16.04.2019
Selected : 16.04.2019
Labels conflict with word 16.04.2019 --- B-DOCNUM : 1; B-DOCDATE : 16.04.2019
Selected : 16.04.2019
Labels conflict with word 10.08.2019 --- B-DOCNUM : 1; B-DOCDATE : 10.08.2019
Selected : 10.08.2019
Labels conflict with word 10.08.2019 --- B-DOCNUM : 1; B-DOCDATE : 10.08.2019
Selected : 10.08.2019
Labels conflict with word 19.03.2019 --- B-DOCNUM : 1; B-DOCDATE : 19.03.2019
Selected : 19.03.2019
Labels conflict with word 19.03.2019 --- B-DOCNUM : 1; B-DOCDATE : 19.03.2019
Selected : 19.03.2019
Labels conflict with word 18.04.2018 --- B-DOCNUM : 1; B-DOCDATE : 18.04.2018
Selected : 18.04.2018
Labels conflict with word 15 --- B-DOCNUM : 1; B-DOCDATE : 15 мая 2019
Selected : 15 мая 2019
Labels conflict with word 15 --- B-DOCNUM : 1; B-DOCDATE : 15 мая 2019
Selected : 15 мая 2019
Labels conflict with word 16 --- B-DOCNUM : 1; B-DOCDATE : 16 мая 2019
Selected : 16 мая 2019
Labels conflict wi

In [13]:
df = d_merged.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})

In [14]:
df.head()

,id,tokens,ner_tags
0,SV990125172_ОРГ__INM_18'09'58_1.pdf,"[l, *, универсальный, приложение, №, 1, постан...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-D..."
1,SS528581_ОРГ__INM_17'05'39_A.pdf,"[l, лист, согласования, стр., 1, 1, 2, 9, *, 0...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,SF435712_ОРГ__INM_15'25'41_A.pdf,"[l, приложение, №, постановлению, правительств...","[O, O, O, O, O, O, O, O, O, B-DOCNUM, B-DOCDAT..."
3,SV990124943_ОРГ__INM_19'00'43_A.pdf,"[l, акционерное, общество, регистраторское, об...","[O, O, O, O, O, B-DOCCPTY, O, O, O, O, O, O, O..."
4,S29013419_ОРИГИНАЛ__IND_1_22.05.12'21'36.pdf,"[l, *, 000стандарт, безопасности, адрес:, 1973...","[O, O, O, B-DOCCPTY, O, O, O, O, O, O, O, O, O..."


In [15]:
df.shape

(4000, 3)

In [16]:
df.to_csv("NER_PAD_agg.csv", index=False)

In [17]:
all_tags = set()
word_to_ix = {}
for _, row in df.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
    for word in row.tokens:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
all_tags = sorted(list(all_tags))
tag_to_idx = {t: i for i, t in enumerate(all_tags)}
label_list = {i: t for i, t in enumerate(all_tags)}

In [18]:
def find_token(x):
    SUFFIXES = ['_DET', '_NOUN', '_VERB', '_ADJ', '']
    res = []
    for w in x['tokens']:
        has_found = False
        for suffix in SUFFIXES:
            if w + suffix in word_to_ix.keys():
                res.append(word_to_ix[w + suffix])
                has_found = True
                break
        if not has_found:
            res.append(0)
    return res

df['encoded_ner_tags'] = df.apply(lambda x: [tag_to_idx[t] for t in x['ner_tags']], axis=1)
df['int_tokens'] = df.apply(find_token, axis=1)

In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
!pip install keras-crf
!pip install tensorflow==1.15.0 keras==2.2.4

     |████████████████████████████████| 686kB 5.7MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=d94d5111e42817b1814b36d4d6e639640de0fcced9f9a96932662c19a4c5ffcd
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 412.3MB 44kB/s 
     |████████████████████████████████| 317kB 34.7MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 3.8MB 24.8MB/s 
     |████████████████████████████████| 512kB 32.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=be311fb7dbfbbee2dbf2addeea3a0b12414e12cd437894a19713df9cd122640f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirem

In [21]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-6337w6_n
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-6337w6_n
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=7434cb65652205ef08fbd729fbcca179acef1b287d03ef57996fc7e7d329e9d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-f0fc51_s/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [22]:
import tensorflow as tf

def focal_loss(gamma=2., alpha=1.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [23]:
from keras.optimizers import Adam
from keras.models import *
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy, crf_marginal_accuracy

MAX_WORDS = len(word_to_ix)
EMBEDDING_LENGTH = 1024
MAX_SEQUENCE_LENGTH = np.max(df['int_tokens'].apply(len))
HIDDEN_SIZE = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_LENGTH, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True)))
# model.add(Dense(50))
model.add(TimeDistributed(Dense(len(label_list), activation="relu")))
crf = CRF(len(label_list), learn_mode='marginal')
model.add(crf)

model.compile(Adam(lr=0.005), loss=focal_loss(), metrics=[crf_marginal_accuracy])
model.summary()

Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 943, 1024)         68114432  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 943, 200)          900000    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 943, 16)           3216      
_________________________________________________________________
crf_1 (CRF)                  (None, 943, 16)           560       
Total params: 69,018,208
Trainable params: 69,018,208
Non-trainable params: 0
_________________________________________________________________


In [26]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


callbacks = [EarlyStopping(monitor='val_crf_marginal_accuracy', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_crf_marginal_accuracy', save_best_only=True)]

x_train = pad_sequences(train['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH)
y_train = pad_sequences(train['encoded_ner_tags'], padding='post', maxlen=MAX_SEQUENCE_LENGTH, value=tag_to_idx['O'])
y_train = to_categorical(y_train, num_classes=len(label_list))
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=20,
    batch_size=200,
    callbacks=callbacks,
    validation_split=0.1,
)

model.load_weights('best_model.h5')





Train on 2880 samples, validate on 320 samples
Epoch 1/20





2880/2880 [==============================] - 105s 36ms/step - loss: 0.6851 - crf_marginal_accuracy: 0.5067 - val_loss: 0.3263 - val_crf_marginal_accuracy: 0.8710
Epoch 2/20
2880/2880 [==============================] - 98s 34ms/step - loss: 0.2042 - crf_marginal_accuracy: 0.9530 - val_loss: 0.1596 - val_crf_marginal_accuracy: 0.9806
Epoch 3/20
2880/2880 [==============================] - 98s 34ms/step - loss: 0.0897 - crf_marginal_accuracy: 0.9903 - val_loss: 0.1510 - val_crf_marginal_accuracy: 0.9941
Epoch 4/20
2880/2880 [==============================] - 98s 34ms/step - loss: 0.0532 - crf_marginal_accuracy: 0.9967 - val_loss: 0.1637 - val_crf_marginal_accuracy: 0.9962
Epoch 5/20
2880/2880 [==============================] - 97s 34ms/step - loss: 0.0358 - crf_marginal_accuracy: 0.9981 - val_loss: 0.1765 - val_crf_marginal_accuracy: 0.9963
Epoch 6/20
2880/2880 [==============================] - 98s 34ms/step - loss: 0.027

In [29]:
from sklearn_crfsuite import metrics

preds = model.predict(pad_sequences(test['int_tokens'], padding='post', maxlen=MAX_SEQUENCE_LENGTH))
tag_preds = [np.argmax(pred, axis=1) for pred in preds]
truncated_preds = []
for i in range(len(tag_preds)):
    truncated_preds.append(tag_preds[i][:len(test['encoded_ner_tags'].iloc[i])])
print(metrics.flat_classification_report(test['encoded_ner_tags'], truncated_preds, labels=np.arange(14), digits=3))

              precision    recall  f1-score   support

           0      0.812     0.547     0.653       150
           1      0.895     0.380     0.533       179
           2      0.904     0.516     0.657      1243
           3      0.913     0.815     0.861       595
           4      0.995     0.766     0.866       282
           5      0.969     0.969     0.969       609
           6      0.947     0.775     0.853       645
           7      0.859     0.570     0.685       651
           8      0.732     0.682     0.706        88
           9      0.000     0.000     0.000         7
          10      0.827     0.410     0.548       620
          11      0.912     0.828     0.868       564
          12      0.948     0.948     0.948       462
          13      0.927     0.922     0.925       605

   micro avg      0.915     0.706     0.797      6700
   macro avg      0.831     0.652     0.719      6700
weighted avg      0.908     0.706     0.782      6700



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
